In [ ]:
# Import your predictor
from linear_slope_predictor import LinearSlopePredictor
# Example: from PREDICTOR_FILE import PREDICTOR_CLASS

# Add your predictor to the dictionary
predictors = {
    "LinearSlopePredictor": LinearSlopePredictor(),
    # Example: "PREDICTOR_CLASS": PREDICTOR_CLASS_CONSTRUCTOR_CALL
}

In [ ]:
import pandas as pd

raw_data = pd.read_csv("combined_data.csv")

